In [ ]:
!pip install -U googlemaps

  Created wheel for googlemaps: filename=googlemaps-4.6.0-py3-none-any.whl size=38554 sha256=cbefcf84400fcaf022c3b4f974cf7ea92e2ac458ee28d5cdc6a6a2f8c0e82804
  Stored in directory: /root/.cache/pip/wheels/80/db/c0/6d958585fa97b20e250bf437acf7e6e715b4809c2dd4e55367
Successfully built googlemaps


In [ ]:
import time
import googlemaps # pip install googlemaps
import pandas as pd # pip install pandas


API_KEY = 'INSERT YOUR GOOGLE API KEY HERE'
map_client = googlemaps.Client(API_KEY)

address = 'INSERT YOUR DESTINATION ADDRESS HERE'
geocode = map_client.geocode(address=address)
(lat, lng) = map(geocode[0]['geometry']['location'].get, ('lat', 'lng'))


search_string = 'INSERT STORE NAME HERE'
distance = 8000 #this represents the eps-range of the search query in meters
business_list = []

response = map_client.places_nearby(
    location=(lat, lng),
    keyword=search_string,
    radius=distance
)   

business_list.extend(response.get('results'))
next_page_token = response.get('next_page_token')

while next_page_token:
    time.sleep(2)
    response = map_client.places_nearby(
        location=(lat, lng),
        keyword=search_string,
        radius=distance,
        page_token=next_page_token
    )   
    business_list.extend(response.get('results'))
    next_page_token = response.get('next_page_token')

df = pd.DataFrame(business_list)
df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']
#df.to_excel('{0}.xlsx'.format(search_string), index=False)

In [ ]:
'''
Storing data within a dictionary of the following structure:
key: place_id
value: [lat, lng, dist]
'''

#convert to dictionary of structure sketched in the preembel of this cell (key:, ...,value:...)
def extractPID_Lat_Lon(pdframe):
  #init dictionary:
  candidate_loc_dic = {}
  pdframe = pdframe.reset_index()  # make sure indexes pair with number of rows
  for index, row in pdframe.iterrows():
    #extract lat, lng
    loc_lat = ((row['geometry'])['location'])['lat']
    loc_lng = ((row['geometry'])['location'])['lng']
    loc_id = row['place_id']
    candidate_loc_dic[loc_id] = [loc_lat, loc_lng] 
  return candidate_loc_dic

extracted_dict = extractPID_Lat_Lon(df)


In [ ]:

'''
Code originally from: https://stackoverflow.com/questions/58053077/get-distance-from-google-cloud-maps-directions-api

also on shortest routes in google maps: https://stackoverflow.com/questions/18574496/google-distance-matrix-json-shortest-path-php

Input: takes two coordinates/locations
Output: the shortest path/distance from all possible routes between the two coordinates proposed by google
'''

# Import libraries
import googlemaps
from datetime import datetime
import numpy as np


def get_shortest_distance(coordStart, coordDestination):
  #transform format of coordinates (lat, lng) to 'lat,long'
  coords_0 = str(coordStart[0])+','+str(coordStart[1])
  coords_1 = str(coordDestination[0])+','+str(coordDestination[1])

  # Request directions
  now = datetime.now()
  directions_result = map_client.directions(coords_0, coords_1, mode="driving", departure_time=now, alternatives = True)

  #stores distances of all paths from start to destination
  distances_array = []

  for path in directions_result:
    # Get distance
    distance = 0
    legs = path.get("legs")
    for leg in legs:
      distance = (distance + leg.get("distance").get("value"))/1000.0
    distances_array.append(distance)
  mindist = np.min(distances_array)
  #print(distances_array)
  return mindist

In [ ]:
'''
routine takes the dictionary with place_id as key and appends the shortest distance from a starting point to the destination in its value array: [lat, lng, dist]
'''

def append_shortest_distances_to_querydict(querydict, startpos):
  for key in querydict.keys():
    destpos = ((querydict[key])[0],(querydict[key])[1])
    shortestdist = get_shortest_distance(startpos, destpos)
    querydict[key].append(shortestdist)



In [ ]:
'''
routine computes the duration of the /fastest/ path and returns the duration in minutes

Code originally from: https://stackoverflow.com/questions/14635926/google-maps-api-travel-time-with-current-traffic


Input: takes two coordinates/locations
Output: the fastest path from all possible routes between the two coordinates proposed by google
'''


# Import libraries
import googlemaps
from datetime import datetime
import numpy as np


def get_fastest_path_duration(coordStart, coordDestination):
  #transform format of coordinates (lat, lng) to 'lat,long'
  coords_0 = str(coordStart[0])+','+str(coordStart[1])
  coords_1 = str(coordDestination[0])+','+str(coordDestination[1])

  # Request directions
  now = datetime.now()
  directions_result = map_client.directions(coords_0, coords_1, mode="driving", departure_time=now, alternatives = True)

  #stores durations of all paths from start to destination
  durations_array = []

  for path in directions_result:
    # Get distance
    duration = 0
    legs = path.get("legs")
    for leg in legs:
      duration = (duration + leg.get("duration_in_traffic").get("value"))/60.0
    durations_array.append(duration)
  mindur = np.min(durations_array)
  #print(durations_array)
  return mindur


In [ ]:
'''
routine takes the dictionary with place_id as key and appends the lowest driving duration from a starting point to the destination in its value array: [lat, lng, dur]
'''

def append_minduration_path_to_querydict(querydict, startpos):
  for key in querydict.keys():
    destpos = ((querydict[key])[0],(querydict[key])[1])
    mindurdist = get_fastest_path_duration(startpos, destpos)
    querydict[key].append(mindurdist)



In [ ]:
!pip install --upgrade git+https://github.com/m-wrzr/populartimes

  Cloning https://github.com/m-wrzr/populartimes to /tmp/pip-req-build-z5xknpfs
  Running command git clone -q https://github.com/m-wrzr/populartimes /tmp/pip-req-build-z5xknpfs
     |████████████████████████████████| 118 kB 26.1 MB/s 
  Created wheel for populartimes: filename=populartimes-2.0-py3-none-any.whl size=13551 sha256=eb8672da3ea8f5a374dc2cc8c974b43277bcb29687e0de1a6a50996d4e9bbdbe
  Stored in directory: /tmp/pip-ephem-wheel-cache-vw9dnetj/wheels/94/ce/3d/b7e3c37d5c3375d6435ff071b662d5f0b772607154c41daf9b
Successfully built populartimes
  Attempting uninstall: geopy
    Found existing installation: geopy 1.17.0
    Uninstalling geopy-1.17.0:
      Successfully uninstalled geopy-1.17.0


In [ ]:
'''
routine takes a place_id
it returns the current popularity "current_popularity" -> normalize by dividing the popular time value (i.e. 33) by the mean value on that particular week day (i.e. 28).
A value >1 indicates that this place is busier than "on average" at those week days, while a value close to 0 means that this place is barely visited right now contrary to the "average" busy time

get_id(api_key, place_id)
    retrieves the CURRENT popularity for a given place
    :param api_key:
    :param place_id:
    :return: see readme
'''

import populartimes
#for getting weekday
import datetime
import numpy


def compute_relative_popularTimes(place_id):
  #fetch current weekday as string
  weekdaymap = {0 : 'Monday', 1 : 'Tuesday', 2 : 'Wednesday', 3 : 'Thursday', 4 : 'Friday', 5 : 'Saturday', 6 : 'Sunday'}
  currweekday = weekdaymap[datetime.datetime.today().weekday()]

  if('current_popularity' in populartimes.get_id(API_KEY, place_id).keys()):
    #current popularity of that place
    curr_pop = populartimes.get_id(API_KEY, place_id)['current_popularity']
    #average popularity on that day for that place
    day_avg_pop = []
    #fetch the array with the data of popular times for one location
    populartimesarray = populartimes.get_id(API_KEY, place_id)['populartimes']
    if populartimesarray:
      for day in populartimesarray:
        #fetch the data for the current weekday, returns an array of populartimes over each hour of the current weekday
        if day['name'] == currweekday:
          day_avg_pop = day['data']

    #compute for the current weekday populartimes its average
    daily_avg_pop = None
    if day_avg_pop:
      daily_avg_pop = np.average(day_avg_pop)
  
    #get current hour
    now = datetime.datetime.now()
    currhour = now.hour + 1
    currhourpop = day_avg_pop[currhour]
    #compute relative current popularity by dividing the current popularity by the average popularity of the current day
    comp2avg_pop = None
    if (daily_avg_pop != None) and (curr_pop != None):
      comp2avg_pop = curr_pop / currhourpop

    return comp2avg_pop
  else:
    return numpy.inf
  


In [ ]:
'''
routine takes the dictionary with place_id as key and appends the relative popular time of the day: [lat, lng, dur, relpop]
'''

def append_relpopularity_to_querydict(querydict):
  for key in querydict.keys():
    relpop = compute_relative_popularTimes(key)
    querydict[key].append(relpop)

In [ ]:
'''
initial cell for timeline data analysis
data stored in google drive /timelinedata
json files for each month over year(s)
'''

import json
from datetime import date
import glob

#supermarket timeline dictionary
timelinedict = {'Aldi' : [],
                'Rewe' : [], 
                'Lidl' : [], 
                'Edeka' : [], 
                'Famila' : [], 
                'real' : [], 
                'Netto' : [], 
                'Norma' : [], 
                'Rossmann' : [], 
                'dm' : [], 
                'Kaufland' : [], 
                'Marktkauf' : [], 
                'Action' : [], 
                'Markant' : [], 
                'Penny' : [], 
                'V-Markt' : [], 
                'Hit' : []}



'''
store in timeline-dictionary:

first access each file, iterate through and fetch per grocery store/supermarket (keys)
the confidence (should be 60% or above) then store the timestamp in an array (values).
through this we can cover:
(1) the overall frequency how often which supermarket was visited
(2) the recency, which supermarket was visited how many days ago

regarding (1) we cut later on the dictionary based on the top-k most frequently visited supermarkets
'''

#routine for a single json file in a folder of timeline data

def populateTimelineDict(singlejson):
  #json files in a folder named timelinedata of the form YEAR_MONTH.json, e.g. 2022_APRIL.json
  with open(singlejson, 'r') as f:
    data = json.load(f)

  for i in range(len(data['timelineObjects'])):
    if 'placeVisit' in data['timelineObjects'][i].keys():

      #check if location has a name...
      if 'name' in data['timelineObjects'][i]['placeVisit']['location'].keys():

        #timelineObjects --> entry index (0,1,2...,n) --> placeVisit --> location --> name
        #remark on name: check if name from supermarktlist is partially in the location name, e.g. aldi in list is in aldi kronshagen in timeline name; python substring/contains function
        locationname = data['timelineObjects'][i]['placeVisit']['location']['name']

        #timelineObjects --> entry index (0,1,2...,n) --> placeVisit --> location --> locationConfidence (should be >=60)
        locationconfidence = data['timelineObjects'][i]['placeVisit']['location']['locationConfidence']

        #timelineObjects --> entry index (0,1,2...,n) --> placeVisit --> duration --> endTimestamp (capture only YYYY-MM-DD)
        lasttimevisited = date.fromisoformat(data['timelineObjects'][i]['placeVisit']['duration']['endTimestamp'][:10])

        #check if name of supermarket is at least partially in any key of the timeline dictionary...
        for key in timelinedict.keys():
          #split locationname by whitespace and take first string, since that contains the supermarket name, make sure that cases don't matter
          #by making both, dictionary key and locationname to lowercase
          locnameadapted = ((locationname.split())[0]).lower()
          keyadapted = key.lower()
          if locnameadapted == keyadapted and locationconfidence >= 60:
            timelinedict[key].append(lasttimevisited)



In [ ]:
'''
Routine to prune timeline dictionary to the top 5 / most frequently visited places
'''
def pruneTimelineDict(tldict,topk):
  #get first all frequencies sorted in descending order
  frequencyarray = []
  for key in tldict:
    freq = len(tldict[key])
    frequencyarray.append(freq)
  frequencyarray.sort(reverse=True)

  #then access at position topk-1 use that value for pruning
  pruningthreshold = frequencyarray[topk-1]

  #start pruning for all entries in the tldict that have a value array length < pruningthreshold
  for key in list(tldict):
    if len(tldict[key]) < pruningthreshold:
      del tldict[key]



'''
Function retrieves for -one- supermarket location in a 2 km radius other supermarkets (need to remove the supermarket location (start point) itself)
and stores their names in an array
'''
def getNearbySupermarkets(lat, lng):
  search_query = 'supermarket'
  distance = 2000 #this represents the eps-range of the search query in meters
  business_list = []

  response = map_client.places_nearby(
      location=(lat, lng),
      keyword=search_query,
      radius=distance
  )   

  business_list.extend(response.get('results'))
  next_page_token = response.get('next_page_token')

  while next_page_token:
      time.sleep(2)
      response = map_client.places_nearby(
          location=(lat, lng),
          keyword=search_query,
         radius=distance,
         page_token=next_page_token
      )   
      business_list.extend(response.get('results'))
      next_page_token = response.get('next_page_token')

  df = pd.DataFrame(business_list)
  df['url'] = 'https://www.google.com/maps/place/?q=place_id:' + df['place_id']

  #project dataframe down to column of the names of other supermarkets and convert it to a np array
  df_names = df['name'].values

  #remove names after the store, e.g. Aldi Nord --> remove 'Nord'
  singular_store_names = []
  for e in df_names:
    str_arr = e.split()
    singular_store_names.append(str_arr[0])
  
  #transform to set and back to remove duplicates
  nn_stores = list(set(singular_store_names))

  #remove original query / supermarket from array
  for e in nn_stores:
    if e.lower() == search_string.lower():
      nn_stores.remove(e)

  #now store only the supermarkets that are among the top-k ones based on the timeline
  final_popular_markets = []
  popularmarkets = timelinedict.keys()
  for e in nn_stores:
    for f in popularmarkets:
      if e.lower() == f.lower():
        final_popular_markets.append(e)


  return final_popular_markets



'''
aldilat = 54.3541912
aldilng = 10.1050172
aldinndf = getNearbySupermarkets(aldilat, aldilng)
print(aldinndf)
'''


'''
routine goes through all supermarkets locations (keys) of a particular brand and computes per supermarkt an array of in-viccinity supermarkets that are based on the personal timeline top-k visited (value)
'''
def getAllNearestSupermarketsPerSupermarktX(querydict):
  for key in querydict.keys():
    lat, lng, dist, dur, pop = querydict[key]
    popnnmarkets = getNearbySupermarkets(lat, lng)
    querydict[key].append(popnnmarkets)



'''
Routine computes the utility score based on the afore collected information
following things need to be weighted/are terms of the score:
(1) the number of other supermarkets in the neighborhood of radius = eps
(2) per supermarket in neighborhood the relative frequency e.g. 10x lidl / 91 supermarkt visits of the top-k
(3) per supermarket the number of days (until now) since last visit

how to incorporate these three facators?
invert (^-1) to get a 'utility distance' / meaning that the lower the better...?

Example:
query: aldi
in total 2 other supermarkets in radius of 1km:
* rewe; overall 45 times visited, last visit 20 days ago
* edeka; overall 22 times visited, last visit 1 day ago
'''

'''
routine to compute the utility score
call this function with a check in if-clause if viccinityarray is not zero. if it is, set score to zero since there is no 'utility' at all for that query supermarket
'''

def computeUtilityPerQueryLocation(viccinityarray):
  #preliminary: compute total number of visits of top-k supermarkets
  freq_totalvisits = 0
  for supermarket in timelinedict:
    freq_totalvisits = freq_totalvisits + len(timelinedict[supermarket])



  #first term: compute ratio of number of top-k popular supermarkets in viccinity of a query supermarket to the total number (top-k) supermarkets
  #+1 is since we account for the query supermarket contributing to the utility of a place
  term1_topkamount = float(len(viccinityarray)+1) / float(len(timelinedict.keys()))
  


  #second term: compute sum over relations of frequencies per top-k supermarket occuring in viccinity of a query supermarket
  term2_relationsfreq = 0
  for supermarket in viccinityarray:
    #to match supermarket name to the key of timelinedict
    supermarketcaseopt = ''
    for key in timelinedict.keys():
      if supermarket.lower() == key.lower():
        supermarketcaseopt = key

    freq_supermarket = len(timelinedict[supermarketcaseopt])
    term2_relationsfreq = term2_relationsfreq + (float(freq_supermarket) / float(freq_totalvisits))
  #add the own query supermarket to the term
  term2_relationsfreq = term2_relationsfreq + (float(len(timelinedict[search_string])) / float(freq_totalvisits))
  

  #third term: The sum over the inverse of number of days since last visited per supermarket in the viccinity of the query supermarket
  term3_dayssinceratio = 0
  for supermarket in viccinityarray:
    #to match supermarket name to the key of timelinedict
    supermarketcaseopt = ''
    for key in timelinedict.keys():
      if supermarket.lower() == key.lower():
        supermarketcaseopt = key
    

    #get latest date since last visited a particular supermarket 'type'
    lasttime = sorted(timelinedict[supermarketcaseopt])[-1]
    #get number of days between last time visited and today (current day)
    currtime_raw = datetime.datetime.now()
    #truncate currtime_raw by removing time
    currtime = datetime.date(currtime_raw.year, currtime_raw.month, currtime_raw.day)
    deltadays = (currtime - lasttime).days
    term3_dayssinceratio = term3_dayssinceratio + (float(1)/float(deltadays))

  utilityscore = term1_topkamount + term2_relationsfreq + term3_dayssinceratio
  #inverted score, because we want to have the lower values the better, since we also aim for lowest traffic time and lowest distance
  #facilitates later on the computation of the pareto front
  inverted_score = len(timelinedict.keys())+2 - utilityscore
  return inverted_score


In [ ]:
'''
routine takes the dictionary with place_id as key and appends the utility of that location: [lat, lng, dist, dur, poptimes, history, util]
'''

def append_utility_to_querydict(querydict):
  for key in querydict.keys():
    neighborhood_list = querydict[key][5]
    if neighborhood_list:
      util = computeUtilityPerQueryLocation(neighborhood_list)
      querydict[key].append(util)
    else:
      #for the case that no other supermarket is in viccinity of query supermarket --> penalize with highest score possible
      maxscore = len(timelinedict.keys())+2
      querydict[key].append(maxscore)

In [ ]:
#computation of lat,lng coors, shortest distance, minimum traffic duration, relative populartimes

from datetime import datetime

startcoor = (lat, lng)

append_shortest_distances_to_querydict(extracted_dict, startcoor)

append_minduration_path_to_querydict(extracted_dict, startcoor)

import datetime

append_relpopularity_to_querydict(extracted_dict)


alljsonfilesarray = glob.glob('INSERT PATH TO GOOGLE TIMELINE DATA HERE')

for jsonfile in alljsonfilesarray:
  populateTimelineDict(jsonfile)   

pruneTimelineDict(timelinedict, 5)

getAllNearestSupermarketsPerSupermarktX(extracted_dict)

append_utility_to_querydict(extracted_dict)
for key in extracted_dict:
  print('key: ', key, ' val: ',extracted_dict[key])

print(extracted_dict)

key:  ChIJ4-7amiFUskcRx-eGSnsU11k  val:  [54.3002176, 10.0878471, 6.612, 14.85, 0.8627450980392157, ['Lidl', 'Rewe', 'EDEKA', 'REWE'], 5.750335481082384]
key:  ChIJDZ-XVWhWskcR2dwtbNtK8O8  val:  [54.3293103, 10.1336699, 2.938, 7.316666666666666, 0.7945205479452054, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658]
key:  ChIJsQVmc-9WskcRS3BV6SyHPoc  val:  [54.3009059, 10.1422826, 6.389, 16.416666666666668, 1.94, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658]
key:  ChIJs9hxzan4skcRqcd-I9tLT34  val:  [54.36001830000001, 10.1336367, 5.267, 7.533333333333333, 0.703125, ['REWE'], 7.0189393939393945]
key:  ChIJv1foXo1WskcRc46YWdst7G8  val:  [54.30565739999999, 10.126084, 4.824, 12.883333333333333, 0.7910447761194029, ['Rewe', 'EDEKA', 'Lidl', 'REWE'], 5.750335481082384]
key:  ChIJr3BHgGxWskcRTMhzuUNuFCs  val:  [54.33495600000001, 10.1320073, 2.985, 7.733333333333333, 1.0158730158730158, ['EDEKA', 'Lidl', 'REWE'], 6.366812753809658]
key:  ChIJbdJh06xVskcRmTMLU8MGjxw  val:  [54.3237325, 10.054179, 5

In [ ]:
#project down to columns containing place_id and vicinity for label information in the plot
df_projected = df[['place_id', 'vicinity']]

df_projected.head()

df_dict = df_projected.set_index('place_id').T.to_dict('list')

df_dict